## Preparing news dataset ##

In [1]:
import numpy as np
import pandas as pd

json_files = [
    "/kaggle/input/my-private-datasets/graph_news/tg_chatexport__rian_ru__20181018-20201018.json",
    "/kaggle/input/my-private-datasets/graph_news/tg_chatexport__rian_ru__20201019-20221019.json",
]
# json_filename = "../data/external/tg_chatexport__rian_ru__20220601-20221009.json.zip"

list_of_df = []
for json_filename in json_files:
    list_of_df.append(pd.read_json(json_filename))

df_news = pd.concat(list_of_df).reset_index()
del list_of_df
df_news = df_news["messages"].apply(pd.Series)

# Для текущего ноутбука, оставляем только:
#  - сообщения без доп контента (например видео, фото)
#  - сообщения не являющиеся reply на другие сообщения
#  - в поле text содержится объект типа str, а не list (как в части новостей)
#  - поле текст не пустое
#  - дата сообщения и текст
df_news = df_news[(df_news.file.isnull() & 
                   df_news.photo.isnull() &
                   df_news.reply_to_message_id.isnull() & 
                   (df_news.text.map(type) == str) &
                   (df_news.text != ''))]
df_news = df_news[["date", "text"]]
df_news

,date,text
0,2019-07-08T09:26:45,"Привет, друзья. Мы возвращаемся в Telegram! Вк..."
3,2019-07-08T11:00:56,❗️Число жертв паводка в Иркутской области возр...
4,2019-07-08T11:37:15,⚡️Иран превысил предусмотренный ядерной сделко...
6,2019-07-08T12:32:39,❗️Зеленский предложил Путину провести встречу ...
15,2019-07-08T15:29:25,❗️Генпрокуратура Украины возбудила дело против...
...,...,...
171295,2022-10-19T12:52:13,"Эрдоган заявил, что договорился с Путиным по г..."
171298,2022-10-19T13:24:19,"31% россиян считают, что православная церковь ..."
171299,2022-10-19T13:28:01,Воздушная тревога объявлена на всей территории...
171300,2022-10-19T13:29:19,Причин для эвакуации населения Запорожской обл...


## Summarization (batch inference / preprocessing dataset) ##

In [2]:
import re
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


def inference(texts, model, tokenizer, 
               tokenizer_kwargs = {}, 
               generate_kwargs = {}, 
               num_beams=5,):
        
    input_ids = tokenizer(
        texts,
        return_tensors="pt",
        **tokenizer_kwargs
    ).to(device)
    
    output_ids = model.generate(
        **input_ids,
        num_beams=num_beams,
        **generate_kwargs
    )

    summary = tokenizer.batch_decode(
        output_ids,
        skip_special_tokens=True,
    )
    return summary

In [3]:
# for unit-test
# df_news = df_news.sample(330)

In [4]:
experiment = {
        "experiment_name": "IlyaGusev__rut5_base_headline_gen_telegram",
        "model_name": "IlyaGusev/rut5_base_headline_gen_telegram",
        "text_prefix": "",
        "tokenizer_kwargs": {"max_length": 600,"padding": True, "truncation": True, "add_special_tokens": True},
        "generate_kwargs": {}
    }

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

dataloader = DataLoader(df_news.text.to_list(), batch_size=20)

tokenizer = AutoTokenizer.from_pretrained(experiment["model_name"])

model = AutoModelForSeq2SeqLM.from_pretrained(experiment["model_name"])
model.to(device);

Downloading:   0%|          | 0.00/327 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/808k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/766 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/932M [00:00<?, ?B/s]

In [5]:
results = []
for batch in tqdm(dataloader):
    results.extend(inference(batch, model, tokenizer, 
                             experiment["tokenizer_kwargs"], 
                             experiment["generate_kwargs"]))

df_news[experiment["experiment_name"]] = pd.Series(results, index=df_news.index)
df_news

100%|██████████| 2955/2955 [1:12:04<00:00,  1.46s/it]


,date,text,IlyaGusev__rut5_base_headline_gen_telegram
0,2019-07-08T09:26:45,"Привет, друзья. Мы возвращаемся в Telegram! Вк...","Привет, друзья! Включайте уведомления и следит..."
3,2019-07-08T11:00:56,❗️Число жертв паводка в Иркутской области возр...,Число жертв паводка в Иркутской области возрос...
4,2019-07-08T11:37:15,⚡️Иран превысил предусмотренный ядерной сделко...,Иран превысил предусмотренный ядерной сделкой ...
6,2019-07-08T12:32:39,❗️Зеленский предложил Путину провести встречу ...,Зеленский предложил Путину провести встречу в ...
15,2019-07-08T15:29:25,❗️Генпрокуратура Украины возбудила дело против...,ГПУ возбудила дело против канала News One из-з...
...,...,...,...
171295,2022-10-19T12:52:13,"Эрдоган заявил, что договорился с Путиным по г...",Эрдоган: Европа может использовать российский ...
171298,2022-10-19T13:24:19,"31% россиян считают, что православная церковь ...","Большинство россиян считают, что РПЦ слишком а..."
171299,2022-10-19T13:28:01,Воздушная тревога объявлена на всей территории...,Воздушная тревога объявлена на всей территории...
171300,2022-10-19T13:29:19,Причин для эвакуации населения Запорожской обл...,В Запорожской области нет причин для эвакуации...


In [6]:
df_news.to_csv("news_with_summarization.csv")
df_news.to_excel("news_with_summarization.xlsx")